# Extração de Dados de Desmatamento - MapBiomas
Este notebook coleta e processa dados de desmatamento para os municípios da Serra do Penitente usando a API do MapBiomas Alerta.

In [13]:
# Importar bibliotecas necessárias
import requests
import pandas as pd
import os

In [14]:
# Criar diretório para salvar saída
os.makedirs("data/generated", exist_ok=True)

In [15]:
# Municípios da Serra do Penitente
municipios_alvo = ["Balsas", "Tasso Fragoso", "Alto Parnaíba"]

In [16]:
# Endpoint da API MapBiomas Alerta
api_url = "https://plataforma.alerta.mapbiomas.org/api/v1/alert/relatoryAlert"

In [17]:
# Função para buscar dados de um município com tratamento de erros
def buscar_dados_municipio(municipio_nome):
    params = {
        "state": "MA",  # Maranhão
        "municipality": municipio_nome,
        "startYear": 2000,
        "endYear": 2023,
        "coverageType": "Vegetação Nativa"
    }
    response = requests.get(api_url, params=params)
    if response.status_code == 200:
        try:
            return response.json()
        except requests.JSONDecodeError:
            print(f"Erro ao decodificar JSON para o município {municipio_nome}. Resposta: {response.text}")
            return []
    else:
        print(f"Erro ao buscar dados para {municipio_nome}. Código de status: {response.status_code}. Resposta: {response.text}")
        return []

In [18]:
# Coletar dados
dados_gerais = []
for municipio in municipios_alvo:
    dados = buscar_dados_municipio(municipio)
    for alerta in dados:
        dados_gerais.append({
            "municipio": municipio,
            "ano": alerta.get("year"),
            "area_desmatada_ha": alerta.get("deforestationAreaHa", 0)
        })

Erro ao decodificar JSON para o município Balsas. Resposta: <!doctype html><html lang="pt-BR"><head><meta charset="utf-8"/>
<script type="text/javascript">window.NREUM||(NREUM={});NREUM.info={"beacon":"bam.nr-data.net","errorBeacon":"bam.nr-data.net","licenseKey":"NRJS-cd5ab77247638685b1d","applicationID":"981055361","transactionName":"Jg0IRUtaClVXFEkRFQQWD1IWXAhdVx4=","queueTime":0,"applicationTime":4,"agent":""}</script>
<script type="text/javascript">(window.NREUM||(NREUM={})).init={privacy:{cookies_enabled:true},ajax:{deny_list:["bam.nr-data.net"]},distributed_tracing:{enabled:true}};(window.NREUM||(NREUM={})).loader_config={agentID:"1120061708",accountID:"3470651",trustKey:"3470651",xpid:"VwIAUVBWCRAJXFBSAgQEUFU=",licenseKey:"NRJS-cd5ab77247638685b1d",applicationID:"981055361"};;/*! For license information please see nr-loader-spa-1.288.1.min.js.LICENSE.txt */
(()=>{var e,t,r={8122:(e,t,r)=>{"use strict";r.d(t,{a:()=>i});var n=r(944);function i(e,t){try{if(!e||"object"!=typeof e)r

In [19]:
# Criar DataFrame consolidado
df_desmatamento = pd.DataFrame(dados_gerais)

In [20]:
# Agrupar área total desmatada por município e ano
df_final = df_desmatamento.groupby(["municipio", "ano"], as_index=False).sum()

KeyError: 'municipio'

In [ ]:
# Salvar dataset
output_path = "data/generated/desmatamento_serra_penitente.csv"
df_final.to_csv(output_path, index=False)
print(f"✅ Dataset de desmatamento gerado: {output_path}")
print(df_final.head())